### Step1. Create the MLflow Experiment

In [49]:
import mlflow
experiment_id = mlflow.create_experiment("MLFlow_Experiment")

##### 1.1.Fetch the Experiment ID

In [50]:
experiment = mlflow.get_experiment(experiment_id)

##### 1.2.Set the Tracking URI and print the Experiment Details

In [51]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
experiment = mlflow.get_experiment(experiment_id)
print("Name: {}".format(experiment.name))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))
print("Experiment ID: {}".format(experiment.experiment_id))

Name: MLFlow_Experiment
Artifact Location: ./artifacts/1
Lifecycle_stage: active
Experiment ID: 1


### Step2. Import required libraries

In [52]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
import mlflow
import mlflow.xgboost

### Step3. Print the MLflow Version, Tracking URI, Experiment Details

In [61]:
print("MLflow Version:", mlflow.__version__)
print("MLflow Tracking URI:", mlflow.get_tracking_uri())
print("XGBoost version:",xgb.__version__)
client = mlflow.tracking.MlflowClient()

MLflow Version: 1.26.1
MLflow Tracking URI: http://127.0.0.1:5000/
XGBoost version: 1.6.1


### Step4. Load the Data

In [62]:
data = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/winequality-white.csv")

### Step5. Train Test Split

In [63]:
train, test = train_test_split(data, test_size=0.2, random_state=2022)

In [64]:
X_train = train.drop("quality", axis=1)
X_test = test.drop("quality", axis=1)

In [65]:
y_train = train["quality"]
y_test = test["quality"]

### Step6. Define Model Parameters

In [72]:
params = {
    'max_depth': 6,
    'min_child_weight': 1,
    'random_state': 2022
}

### Step7. Define the Model and Register Model with `mlflow.start_run()`

In [75]:
with mlflow.start_run(experiment_id=experiment.experiment_id, run_name='xgboost') as run:
    run_id = run.info.run_id
    experiment_id = run.info.experiment_id
    
    print("MLFlow:")
    print(" Run ID:", run_id)
    print(" Experiment ID:", experiment_id)
    print(" Experiment Name:", client.get_experiment(experiment_id).name)
    
    # MLflow parameters
    print("MLflow Parameters:")
    print(" - max_depth:", params['max_depth'])
    print(" - min_child_weight:", params['min_child_weight'])
    
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    
    # MLflow Artifacts
    prediction = model.predict(X_test)
    print("Prediction:", prediction)
    rmse = np.sqrt(mean_squared_error(y_test, prediction))
    mse = mean_squared_error(y_test, prediction)
    r2 = r2_score(y_test, prediction)
    print("Metrics :")
    print("- RMSE:", rmse)
    print("- MSE:", mse)
    print("- R2:", r2)
    
    # set tag for the run
    # mlflow.set_tag("run_id", run_id)
    
    # MLflow metrics
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("MaxDepth", params['max_depth'])
    mlflow.log_metric("MinChildWeight", params['min_child_weight'])
    mlflow.log_metric("RandomState", params['random_state'])

MLFlow:
 Run ID: e0b02bcf7d954c589950e07191945d17
 Experiment ID: 1
 Experiment Name: MLFlow_Experiment
MLflow Parameters:
 - max_depth: 6
 - min_child_weight: 1
Prediction: [3.9537404 4.9718857 6.1941724 6.5804787 6.5161133 5.8462634 5.117705
 6.408802  5.477459  6.6296005 6.010759  6.0596    3.919864  6.9541965
 5.060921  5.057375  6.3652515 6.6185803 5.4289618 7.131703  5.426098
 5.9478006 5.9134016 5.3115606 5.8375573 7.066308  5.2387238 4.849651
 6.417327  5.4156723 5.331392  6.0960474 6.0392017 5.6393723 5.9035916
 5.571603  6.1613913 6.087812  4.9416842 6.087812  6.176339  5.93556
 5.283395  4.813785  4.7850347 6.4574194 5.897359  6.124659  6.0697174
 6.0494213 6.2059326 5.0858984 5.7123446 5.920784  5.8042564 6.2164207
 6.4371967 5.9504023 6.0427885 6.5554805 5.662077  6.995541  4.7553344
 5.4481254 5.9489317 6.2349524 7.1644635 5.041433  5.655295  7.0489445
 6.1074786 6.1560483 5.247224  5.495388  6.585176  5.984078  6.0826807
 6.1941733 6.750443  5.191347  5.6313777 5.7548394

### Step9.Model Register Individually

In [78]:
# Log MLflow Model
mlflow.xgboost.log_model(model, "artifect", registered_model_name="XGBoost_Model")

Registered model 'XGBoost_Model' already exists. Creating a new version of this model...
2022/06/14 17:29:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGBoost_Model, version 2
Created version '2' of model 'XGBoost_Model'.


ModelInfo(artifact_path='artifect', flavors={'python_function': {'loader_module': 'mlflow.xgboost', 'python_version': '3.8.8', 'data': 'model.xgb', 'env': 'conda.yaml'}, 'xgboost': {'xgb_version': '1.6.1', 'data': 'model.xgb', 'model_class': 'xgboost.sklearn.XGBRegressor', 'code': None}}, model_uri='runs:/8379513f8f334d1cb2994533ccb6c9d8/artifect', model_uuid='699c1b49ba9d40fd8986d184b688e28f', run_id='8379513f8f334d1cb2994533ccb6c9d8', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-06-14 11:59:07.656577', mlflow_version='1.26.1')

### Step10.Model Prediction Using Register Model

In [90]:
# Predict with MLflow Model
print("Predict with MLflow Model:")
model = mlflow.xgboost.load_model("models:///XGBoost_Model/Production")
print("="*50)
print("Model:\n", model)
print("="*50)
prediction = model.predict(X_test)
# print("Prediction.type:", type(prediction))
# print("="*50)
# print("Prediction.shape:", prediction.shape)
# print("="*50)
print("Prediction:\n", prediction)
print("Prediction Done.")

Predict with MLflow Model:
Model:
 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=2022,
             reg_alpha=0, reg_lambda=1, ...)
Prediction:
 [3.9537404 4.9718857 6.1941724 6.5804787 6.5161133 5.8462634 5.117705
 6.408802  5.477459  6.6296005 6.010759  6.0596    3.919864  6.9541965
 5.060921  5.057375  6.3652515 6.6185803 5.4289618 7.131703  5.426098
 5.9478006 5.9134016 5.3115606 5.8375573 7.066308